# Protocol 4.0
VLS and SAG NWs with standard lock-in technique
<br>
The code can be used for 1, 2 or 3 devices silmutaneously
<br>
This version supports both 4-probe and 2-probe measurements

## Imports

In [ ]:
# Copy this to all notebooks!
from qcodes.logger import start_all_logging
start_all_logging()

# Import qcodes and other necessary packages
import qcodes as qc
import numpy as np
import time
from time import sleep
import matplotlib
import matplotlib.pyplot as plt
import os
import os.path


# Import device drivers 
from qcodes.instrument_drivers.QuantumDesign.DynaCoolPPMS import DynaCool
from qcodes.instrument_drivers.Keysight.Infiniium import Infiniium

# Import qcodes packages 
from qcodes import Station
from qcodes import config
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.database import initialise_database,get_DB_location
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment,
                                                 load_experiment_by_name)
from qcodes.instrument.base import Instrument
from qcodes.utils.dataset.doNd import do1d,do2d


%matplotlib notebook
go = 7.7480917310e-5

## Station
(Need to load 3 Keithleys and 6 Lock-In Amps)

In [ ]:
# Create station, instantiate instruments
Instrument.close_all()
path_to_station_file = 'C:/Users/lyn-ppmsmsr-01usr/Desktop/station.yaml'
#                       'file//station.yaml'
# Here we load the station file.
station = Station()
station.load_config_file(path_to_station_file)

# Connect to ppms
#Instrument.find_instrument('ppms_cryostat')
ppms = DynaCool.DynaCool(name = "ppms_cryostat", address="TCPIP0::10.10.117.37::5000::SOCKET")
station.add_component(ppms)

# SRS
lockin_1 = station.load_instrument('lockin_1')
lockin_2 = station.load_instrument('lockin_2')
lockin_3 = station.load_instrument('lockin_3')
lockin_4 = station.load_instrument('lockin_4')
lockin_5 = station.load_instrument('lockin_5')
lockin_6 = station.load_instrument('lockin_6')

# DMMs
dmm_a = station.load_instrument('Keithley_A')
dmm_b = station.load_instrument('Keithley_B')
dmm_c = station.load_instrument('Keithley_C')

dmm_a.smua.volt(0) # Set voltages to 0
dmm_a.smub.volt(0) # Set voltages to 0

dmm_b.smua.volt(0) # Set voltages to 0
dmm_b.smub.volt(0) # Set voltages to 0

dmm_c.smua.volt(0) # Set voltages to 0
dmm_c.smub.volt(0) # Set voltages to 0

for inst in station.components.values():
    inst.print_readable_snapshot()

## DB File, Location

In [ ]:
### Initialize database, make new measurement
mainpath = 'C:/Users/MicrosoftQ/Desktop/Results/Operator_name' #remember to change << /Operator_name >> to save the db file in your own user folder
config.current_config.core.db_location = os.path.join(mainpath,'GROWTHXXXX_BATCHXX_YYYYMMDD.db')
config.current_config
newpath = os.path.join(mainpath,'GROWTHXXXX_BATCHXX_YYYYMMDD') 
if not os.path.exists(newpath):
    os.makedirs(newpath)
figurepath = newpath
initialise_database()

## Functions

In [ ]:
def wait_for_field():
    time.sleep(1)
    Magnet_state = ppms.magnet_state()
    while Magnet_state is not 'holding':
        #print('waiting for field')
        time.sleep(0.1)
        Magnet_state = ppms.magnet_state()
        #print('field ready')
    return

def wait_for_field_ramp():
    Magnet_state = ppms.magnet_state()
    while Magnet_state is not 'ramping':
        time.sleep(1)
        Magnet_state = ppms.magnet_state()
    return

def field_ready():
    return ppms.magnet_state() == 'holding'

def wait_for_temp():
    Temp_state = ppms.temperature_state()
    while Temp_state is not 'stable':
        time.sleep(1)
        Temp_state = ppms.temperature_state()
    return

def wait_for_near_temp():
    Temp_state = ppms.temperature_state()
    while Temp_state is not 'near':
        time.sleep(2)
        Temp_state = ppms.temperature_state()
    time.sleep(10)
    return

## Lock-in add-on functions
Gains and conductance

In [ ]:
# AMPLIFICATIONS AND VOLTAGE DIVISIONS

ACdiv = 1e-4
DCdiv = 1e-2

GIamp1 = 1e7
GVamp2 = 100
GIamp3 = 1e6
GVamp4 = 100
GIamp5 = 1e6
GVamp6 = 100

In [ ]:
# DEFINICTIONS OF FUNCTIONS FOR DIFFERENTIAL CONDUCTANCE AND RERISTANCE FOR 2 AND 4 PROBE MEASUREMENTS
# Lock-ins 1(current), 2(voltage)

def desoverh_fpm12():
    volt_ampl_1 = lockin_1.X
    volt_ampl_2 = lockin_2.X
    
    I_fpm = volt_ampl_1()/GIamp1
    V_fpm = volt_ampl_2()/GVamp2
    if V_fpm== 0:
        dcond_fpm = 0
    else:
        dcond_fpm = I_fpm/V_fpm/go
    return dcond_fpm

def desoverh_tpm1():
    volt_ampl = lockin_1.X
    sig_ampl = lockin_1.amplitude()
    I_tpm = volt_ampl()/GIamp1
    V_tpm = sig_ampl*ACdiv
    dcond_tpm = I_tpm/V_tpm/go
    return dcond_tpm

def ohms_law12():
    volt_ampl_1 = lockin_1.X
    volt_ampl_2 = lockin_2.X
    
    I_fpm = volt_ampl_1()/GIamp1
    V_fpm = volt_ampl_2()/GVamp2
    if I_fpm== 0:
        res_fpm = 0
    else:
        res_fpm = V_fpm/I_fpm
    return res_fpm





# Lock-ins 3(current), 4(voltage)

def desoverh_fpm34():
    volt_ampl_3 = lockin_3.X
    volt_ampl_4 = lockin_4.X
    
    I_fpm = volt_ampl_3()/GIamp3
    V_fpm = volt_ampl_4()/GVamp4
    if V_fpm== 0:
        dcond_fpm = 0
    else:
        dcond_fpm = I_fpm/V_fpm/go
    return dcond_fpm

def desoverh_tpm3():
    volt_ampl = lockin_3.X
    sig_ampl = lockin_3.amplitude()
    I_tpm = volt_ampl()/GIamp1
    V_tpm = sig_ampl*ACdiv
    dcond_tpm = I_tpm/V_tpm/go
    return dcond_tpm

def ohms_law34():
    volt_ampl_3 = lockin_3.X
    volt_ampl_4 = lockin_4.X
    
    I_fpm = volt_ampl_3()/GIamp3
    V_fpm = volt_ampl_4()/GVamp4
    if I_fpm== 0:
        res_fpm = 0
    else:
        res_fpm = V_fpm/I_fpm
    return res_fpm





# Lock-ins 5(current), 6(voltage)

def desoverh_fpm56():
    volt_ampl_5 = lockin_5.X
    volt_ampl_6 = lockin_6.X
    
    I_fpm = volt_ampl_5()/GIamp5
    V_fpm = volt_ampl_6()/GVamp6
    if V_fpm== 0:
        dcond_fpm = 0
    else:
        dcond_fpm = I_fpm/V_fpm/go
    return dcond_fpm

def desoverh_tpm5():
    volt_ampl = lockin_5.X
    sig_ampl = lockin_5.amplitude()
    I_tpm = volt_ampl()/GIamp1
    V_tpm = sig_ampl*ACdiv
    dcond_tpm = I_tpm/V_tpm/go
    return dcond_tpm


def ohms_law56():
    volt_ampl_5 = lockin_5.X
    volt_ampl_6 = lockin_6.X
    
    I_fpm = volt_ampl_5()/GIamp5
    V_fpm = volt_ampl_6()/GVamp6
    if I_fpm== 0:
        res_fpm = 0
    else:
        res_fpm = V_fpm/I_fpm
    return res_fpm

In [ ]:
try:
    lockin_1.add_parameter("diff_conductance_fpm", label="dI/dV", unit="2e^2/h", get_cmd = desoverh_fpm12)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_1.parameters['diff_conductance_fpm']

In [ ]:
try:
    lockin_1.add_parameter("conductance_tpm", label="I/V", unit="2e^2/h", get_cmd = desoverh_tpm1)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_1.parameters['conductance_tpm']

In [ ]:
try:
    lockin_1.add_parameter("resistance_fpm", label="R", unit="Ohm", get_cmd = ohms_law12)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_1.parameters['resistance_fpm']

In [ ]:
try:
    lockin_3.add_parameter("diff_conductance_fpm", label="dI/dV", unit="2e^2/h", get_cmd = desoverh_fpm34)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_3.parameters['diff_conductance_fpm']

In [ ]:
try:
    lockin_3.add_parameter("conductance_tpm", label="I/V", unit="2e^2/h", get_cmd = desoverh_tpm3)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_3.parameters['conductance_tpm']

In [ ]:
try:
    lockin_3.add_parameter("resistance_fpm", label="R", unit="Ohm", get_cmd = ohms_law34)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_3.parameters['resistance_fpm']

In [ ]:
try:
    lockin_5.add_parameter("diff_conductance_fpm", label="dI/dV", unit="2e^2/h", get_cmd = desoverh_fpm56)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_5.parameters['diff_conductance_fpm']

In [ ]:
try:
    lockin_5.add_parameter("conductance_tpm", label="I/V", unit="2e^2/h", get_cmd = desoverh_tpm5)
except KeyError:
    print("parameter already exists. Deleting. Try again")
    del lockin_5.parameters['conductance_tpm']

# Measurement parameters

In [ ]:
Vgmin = -2 #V               [consult the ppt protocol]
Vgmax = +5 #V               [consult the ppt protocol]
Npoints = 801 #             [consult the ppt protocol]


VSD   = 0 #V DC             [consult the ppt protocol]
timedelay = 0.1 # sec       [consult the ppt protocol]


VAC   = 1 #V AC             [consult the ppt protocol]
f     = 136.5 #Hz           [consult the ppt protocol]
tcI   = 0.03  #sec          [consult the ppt protocol]
tcV   = 0.03  #sec          [consult the ppt protocol] Preferably the same with tcI
dB_slope  = 12  # dB        [consult the ppt protocol]

N = 1 #Repetitions          [consult the ppt protocol]

temperature = 1.7 #K
temperature_rate = 0.1

magnetic_field = 0 #T
magnetic_field_rate = 0.22

In [ ]:
# Small calculation for measurement parameters

if 1/f*5 <= tcI and 1/f*5 <= tcV:
    valid_meas  = True
elif 1/f < tcI and 1/f < tcV:
    valid_meas  = True
    print("Warning: Time constant must be much smaller than signal oscillation period", 1/f*1000, "msec")
else:
    valid_meas = False
    print("Error: Time constant must be smaller than signal oscillation period", 1/f*1000, "msec")

if tcI*2.5<=timedelay and tcV*2.5<=timedelay:
    valid_meas  = True
elif tcI<=timedelay and tcV<=timedelay:
    valid_meas  = True
    print("Warning: Time delay is comparable with time constant")
    print("Time constant:",tcI*1e3 ,"msec, (current); ", tcV*1e3, "msec, (voltage)")
    print("Time delay:", timedelay*1e3,"msec")    
else:
    valid_meas = False
    print("Error: Time delay is smaller than the time constant")

valid_meas

## Frequency Test
Small measurement for frequency choise
<br>
Use whichever lock-in you are interested to test (eg. lockin_X)

In [ ]:
new_experiment(name='lockin start-up', sample_name='DEVXX S21D18G38')

# Time constant choise:
# Example: f_min = 60 Hz => t_c = 1/60*2.5 sec = 42 msec => we should choose the closest value: 100 ms
lockin_1.time_constant(0.1)
tdelay = 0.3

dmm_a.smub.output('on')                   # Turn on the gate channel
dmm_a.smub.volt(-2)                       # Set the gate on a very high resistance area (below the pinch-off)

# 1-D sweep for amplitude dependence
#do1d(lockin_1.frequency,45,75,100,tdelay,lockin_1.X,lockin_1.Y,lockin_1.conductance_tpm)

# 2-D sweep repetition on a smaller frequency range for noise inspection
do2d(dmm_a.smua.volt,1,50,50,1,lockin_1.frequency,45,75,100,tdelay,lockin_1.X,lockin_1.Y,lockin_1.conductance_tpm)

dmm_a.smub.volt(0)   
dmm_a.smub.output('off') 

In [ ]:
# Set things up to the station

lockin_1.time_constant(tcI)        # set time constant on the lock-in
lockin_1.frequency(f)              # set frequency on the lock-in
lockin_1.amplitude(VAC)            # set amplitude on the lock-in
lockin_1.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_2.time_constant(tcV)        # set time constant on the lock-in
lockin_2.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_3.time_constant(tcI)        # set time constant on the lock-in
lockin_3.frequency(f)              # set frequency on the lock-in
lockin_3.amplitude(VAC)            # set amplitude on the lock-in
lockin_3.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_4.time_constant(tcV)        # set time constant on the lock-in
lockin_4.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_5.time_constant(tcI)        # set time constant on the lock-in
lockin_5.frequency(f)              # set frequency on the lock-in
lockin_5.amplitude(VAC)            # set amplitude on the lock-in
lockin_5.filter_slope(dB_slope)    # set filter slope on the lock-in

lockin_6.time_constant(tcV)        # set time constant on the lock-in
lockin_6.filter_slope(dB_slope)    # set filter slope on the lock-in

dcond1 = lockin_1.diff_conductance_fpm
cond1 = lockin_1.conductance_tpm
res1   = lockin_1.resistance_fpm
X1  = lockin_1.X
X2  = lockin_2.X
Y1  = lockin_1.Y
Y2  = lockin_2.Y

dcond3 = lockin_3.diff_conductance_fpm
cond3 = lockin_3.conductance_tpm
res3   = lockin_3.resistance_fpm
X3  = lockin_3.X
X4  = lockin_4.X
Y3  = lockin_3.Y
Y4  = lockin_4.Y

dcond5 = lockin_5.diff_conductance_fpm
cond5 = lockin_5.conductance_tpm
res5   = lockin_5.resistance_fpm
X5  = lockin_5.X
X6  = lockin_6.X
Y5  = lockin_5.Y
Y6  = lockin_6.Y

gate   = dmm_a.smub.volt
bias1  = dmm_a.smua.volt
bias3  = dmm_b.smua.volt
bias5  = dmm_b.smub.volt

temp      = ppms.temperature            # read the temperature
temp_set  = ppms.temperature_setpoint   # set the temperature
temp_rate = ppms.temperature_rate       # set the temperature rate


temp_rate(temperature_rate)
temp_set(temperature)


field       = ppms.field_measured        # read the magnetic field
field_set   = ppms.field_target          # set the field; a new qcodes function! field_rate is not in use anymore
field_rate  = ppms.field_rate            # set the the magnetic field rate


field_rate(magnetic_field_rate)
field_set(magnetic_field)

## The measurement

Temperature is considered as control parameter for a sequence of measurements in this cell
<br>
Source-Drain DC bias voltage and gate voltage may applied
<br><b>
This measurement can be used for both WAL and critial field

In [ ]:
# If you want to add bias then uncheck
#dmm_a.smua.output('on') # the bias for 1
#dmm_b.smua.output('on') # the bias for 3
#dmm_b.smub.output('on') # the bias for 5
#bias1(1e-3/DCdiv)
#bias3(1e-3/DCdiv)    
#bias5(1e-3/DCdiv)
    
# If you want to add gate voltage then uncheck
gate(0) # set the gate to zero if you will not apply any
# dmm_a.smub.output('on') # Turn on the gate
#gate(2)

# The control parameter (temperature)
paramrange = [1.7] #arbitrary values
#paramrange = np.arange(0,9,0.5) #steps
#paramrange = np.linspace(0,9,10) #Npoints

# Sweeping parameters
b_start = 0
b_end = 5

for var_param in paramrange:

    ppms.temperature_setpoint(var_param)
    wait_for_temp()
    
    #vv1 = "Vsd1="+"{:.3f}".format(bias1()*DCdiv*1e3)+"mV "
    #vv2 = "Vsd2="+"{:.3f}".format(bias2()*DCdiv*1e3)+"mV "
    #vv2 = "Vsd3="+"{:.3f}".format(bias3()*DCdiv*1e3)+"mV "
    tt = "T="+"{:.3f}".format(temperature())+"K "
    gg = "Vg="+"{:.1f}".format(gate())+"V "
    ff = "f="+"{:.1f}".format(lockin_1.frequency())+"Hz "
    aa = "Ampl="+"{:.4f}".format(lockin_1.amplitude()*ACdiv*1e3)+"mV"
    Conditions = bb + gg + ff + aa
    
    d1 = "/1/ DEV00 S99 VH99 VL99 D99"
    d2 = "/3/ DEV00 S99 VH99 VL99 D99"
    d3 = "/5/ DEV00 S99 VH99 VL99 D99"
    Sample_name = d1# + d2 + d3
    
    Experiment_name = "Protocol 4.0: "
    new_experiment(name=Experiment_name + Conditions, sample_name = Sample_name)
        
    meas = Measurement()
    meas.register_parameter(field)
    meas.register_parameter(dcond1, setpoints=(field,))
    meas.register_parameter(res1, setpoints=(field,))
    meas.register_parameter(X1, setpoints=(field,))
    meas.register_parameter(Y1, setpoints=(field,))
    meas.register_parameter(X2, setpoints=(field,))
    meas.register_parameter(Y2, setpoints=(field,))
#    meas.register_parameter(dcond3, setpoints=(field,))
#    meas.register_parameter(res3, setpoints=(field,))
#    meas.register_parameter(X3, setpoints=(field,))
#    meas.register_parameter(Y3, setpoints=(field,))
#    meas.register_parameter(X4, setpoints=(field,))
#    meas.register_parameter(Y4, setpoints=(field,))
#    meas.register_parameter(dcond5, setpoints=(field,))
#    meas.register_parameter(res5, setpoints=(field,))
#    meas.register_parameter(X5, setpoints=(field,))
#    meas.register_parameter(Y5, setpoints=(field,))
#    meas.register_parameter(X6, setpoints=(field,))
#    meas.register_parameter(Y6, setpoints=(field,))

    field_rate(0.2)
    field_set(b_start)
    ppms.ramp('blocking')
    wait_for_field()
    

    with meas.run() as datasaver:
        run_id = datasaver.run_id
        field_set(b_end)
        field_rate(0.003)
        ppms.ramp('non-blocking')
        while (round(field()*100) != round(b_end*100)):
            datasaver.add_result((field,field()),
                                 (dcond1,dcond1()),(res1,res1()),(X1,X1()),(Y1,Y1()),(X2,X2()),(Y2,Y2()))#,
                                 #(dcond3,dcond3()),(res3,res3()),(X3,X3()),(Y3,Y3()),(X4,X4()),(Y4,Y4()),
                                 #(dcond5,dcond5()),(res5,res5()),(X5,X5()),(Y5,Y5()),(X6,X6()),(Y6,Y6()))

            sleep(timedelay)

dmm_a.smub.output('off')